In [1]:
#Célula 1: Instalações e Importações

!pip install shapash

import pandas as pd
import numpy as np
from category_encoders import OrdinalEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from shapash import SmartExplainer

c:\XAILabProcess\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Célula 2: Reconstrução do Ambiente (Cópia rápida do Notebook 02)

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00601/ai4i2020.csv"
df = pd.read_csv(url)

mapa_nomes = {
    'Type': 'Tipo_Equipamento', 'Air temperature [K]': 'Temp_Ar',
    'Process temperature [K]': 'Temp_Processo', 'Rotational speed [rpm]': 'Rotacao_rpm',
    'Torque [Nm]': 'Torque_Nm', 'Tool wear [min]': 'Desgaste_Ferramenta_min',
    'Machine failure': 'Falha'
}
df = df.rename(columns=mapa_nomes).drop(['UDI', 'Product ID', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], axis=1)

X = df.drop('Falha', axis=1)
y = df['Falha']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

encoder = OrdinalEncoder(cols=['Tipo_Equipamento'])
X_train_enc = encoder.fit_transform(X_train)
X_test_enc = encoder.transform(X_test)

model = XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss')
model.fit(X_train_enc, y_train)

#Instanciando o Shapash novamente
features_dict = {'Temp_Ar': 'Temperatura do Ambiente', 'Rotacao_rpm': 'Velocidade de Rotação', 'Torque_Nm': 'Torque do Motor'}
xpl = SmartExplainer(model=model, preprocessing=encoder, features_dict=features_dict)
xpl.compile(x=X_test_enc)

print("Ambiente restaurado. Iniciando cálculo de métricas de qualidade...")

INFO: Shap explainer type - <shap.explainers._exact.ExactExplainer object at 0x000001E5BF111960>


ExactExplainer explainer: 3001it [00:43, 64.49it/s]                          


Ambiente restaurado. Iniciando cálculo de métricas de qualidade...


In [3]:
#Célula 3: Métrica 1 - Estabilidade (Stability)

#O que mede: Se instâncias muito parecidas recebem a mesma explicação.
#Retorno: Um valor entre 0 e 1. Quanto menor, mais estável (melhor).

stability = xpl.compute_stability_metric(
    selection=list(X_test_enc.index[:100]), # Testando nas primeiras 100 amostras para ser rápido
    max_distance=0.05 # Define o que é "vizinhança" (5% de distância)
)

print(f"\n1. Estabilidade (Stability): {stability:.4f}")
print("   (Interpretação: Mede a variabilidade da explicação em vizinhanças locais. Valores baixos são ideais.)")

AttributeError: 'SmartExplainer' object has no attribute 'compute_stability_metric'

In [ ]:
#Célula 4: Métrica 2 - Consistência (Consistency)

#O que mede: Se as contribuições calculadas pelo SHAP/LIME batem com a importância global das features do modelo.
#Retorno: Um valor entre 0 e 1. Quanto maior, mais consistente (melhor).

consistency = xpl.compute_consistency_metric(
    selection=list(X_test_enc.index[:100])
)

print(f"\n2. Consistência (Consistency): {consistency:.4f}")
print("   (Interpretação: Mede a similaridade entre a importância global e a média das locais. Valores altos indicam coerência.)")

In [ ]:
#Célula 5: Métrica 3 - Compacidade (Compacity)

#O que mede: O quão "resumida" é a explicação.
#Retorno: Percentual de features necessárias para explicar X% da variância da previsão.

compacity = xpl.compute_compacity_metric(
    selection=list(X_test_enc.index[:100]),
    distance=0.90 # Queremos explicar 90% da decisão
)

print(f"\n3. Compacidade (Compacity): {compacity:.4f}")
print("   (Interpretação: Número médio de features necessárias para explicar 90% da previsão.)")